In [1]:
import pandas as pd
import numpy as np
from string_grouper import match_strings, match_most_similar, group_similar_strings, compute_pairwise_similarities, StringGrouper

In [5]:
company_names = 'data/sec__edgar_company_info.csv'
# We only look at the first 50k as an example:
companies = pd.read_csv(company_names)[0:50000]
# Create all matches:
matches = match_strings(companies['Company Name'])
# Look at only the non-exact matches:
matches[matches['left_Company Name'] != matches['right_Company Name']].head()

,left_index,left_Company Name,similarity,right_Company Name,right_index
15,14,"0210, LLC",0.870291,90210 LLC,4211
167,165,1 800 MUTUALS ADVISOR SERIES,0.931615,1 800 MUTUALS ADVISORS SERIES,166
168,166,1 800 MUTUALS ADVISORS SERIES,0.931615,1 800 MUTUALS ADVISOR SERIES,165
172,168,1 800 RADIATOR FRANCHISE INC,1.000000,1-800-RADIATOR FRANCHISE INC.,201
178,173,1 FINANCIAL MARKETPLACE SECURITIES LLC ...,0.949364,"1 FINANCIAL MARKETPLACE SECURITIES, LLC",174


In [6]:
# Create a small set of artificial company names:
duplicates = pd.Series(['S MEDIA GROUP', '012 SMILE.COMMUNICATIONS', 'foo bar', 'B4UTRADE COM CORP'])
# Create all matches:
matches = match_strings(companies['Company Name'], duplicates)
matches

,left_index,left_Company Name,similarity,right_side,right_index
0,12,012 SMILE.COMMUNICATIONS LTD,0.944092,012 SMILE.COMMUNICATIONS,1
1,49777,B.A.S. MEDIA GROUP,0.854383,S MEDIA GROUP,0
2,49855,B4UTRADE COM CORP,1.000000,B4UTRADE COM CORP,3
3,49856,B4UTRADE COM INC,0.810217,B4UTRADE COM CORP,3
4,49857,B4UTRADE CORP,0.878276,B4UTRADE COM CORP,3


In [16]:
# Create a small set of artificial company names:
new_companies = pd.Series(['S MEDIA GROUP', '012 SMILE.COMMUNICATIONS', 'foo bar', 'B4UTRADE COM CORP'], name='New Company')
# Create all matches:
matches = match_most_similar(companies['Company Name'], new_companies, ignore_index=True)
# Display the results:
pd.DataFrame({'new_companies': new_companies, 'duplicates': matches})

,new_companies,duplicates
0,S MEDIA GROUP,B.A.S. MEDIA GROUP
1,012 SMILE.COMMUNICATIONS,012 SMILE.COMMUNICATIONS LTD
2,foo bar,foo bar
3,B4UTRADE COM CORP,B4UTRADE COM CORP


In [12]:
# Add the grouped strings:
companies['deduplicated_name'] = group_similar_strings(companies['Company Name'], ignore_index=True)
# Show items with most duplicates:
companies.groupby('deduplicated_name')['Line Number'].count().sort_values(ascending=False).head(10)

deduplicated_name
ADVISORS DISCIPLINED TRUST                                      1824
AGL LIFE ASSURANCE CO SEPARATE ACCOUNT                           183
ANGELLIST-ART-FUND, A SERIES OF ANGELLIST-FG-FUNDS, LLC          116
AMERICREDIT AUTOMOBILE RECEIVABLES TRUST 2001-1                   87
ACE SECURITIES CORP. HOME EQUITY LOAN TRUST, SERIES 2006-HE2      57
ASSET-BACKED PASS-THROUGH CERTIFICATES SERIES 2004-W1             40
ALLSTATE LIFE GLOBAL FUNDING TRUST 2005-3                         39
ALLY AUTO RECEIVABLES TRUST 2014-1                                33
ANDERSON ROBERT E /                                               28
ADVENT INTERNATIONAL GPE VIII LIMITED PARTNERSHIP                 28
Name: Line Number, dtype: int64

In [17]:
# Create a small set of artificial customer names:
customers_df = pd.DataFrame(
   [
      ('BB016741P', 'Mega Enterprises Corporation'),
      ('CC082744L', 'Hyper Startup Incorporated'),
      ('AA098762D', 'Hyper Startup Inc.'),
      ('BB099931J', 'Hyper-Startup Inc.'),
      ('HH072982K', 'Hyper Hyper Inc.')
   ],
   columns=('Customer ID', 'Customer Name')
).set_index('Customer ID')
# Display the data:
customers_df

,Customer Name
Customer ID,
BB016741P,Mega Enterprises Corporation
CC082744L,Hyper Startup Incorporated
AA098762D,Hyper Startup Inc.
BB099931J,Hyper-Startup Inc.
HH072982K,Hyper Hyper Inc.


In [18]:
# Group customers with similar names:
customers_df[["group-id", "name_deduped"]]  = \
    group_similar_strings(customers_df["Customer Name"])
# Display the mapping table:
customers_df

,Customer Name,group-id,name_deduped
Customer ID,,,
BB016741P,Mega Enterprises Corporation,BB016741P,Mega Enterprises Corporation
CC082744L,Hyper Startup Incorporated,CC082744L,Hyper Startup Incorporated
AA098762D,Hyper Startup Inc.,AA098762D,Hyper Startup Inc.
BB099931J,Hyper-Startup Inc.,AA098762D,Hyper Startup Inc.
HH072982K,Hyper Hyper Inc.,HH072982K,Hyper Hyper Inc.


In [27]:
companies = pd.read_csv(company_names)

In [28]:
# Create a new StringGrouper
string_grouper = StringGrouper(companies['Company Name'], ignore_index=True)
# Check if the ngram function does what we expect:
string_grouper.n_grams('McDonalds')

['mcd', 'cdo', 'don', 'ona', 'nal', 'ald', 'lds']

In [ ]:
# Now fit the StringGrouper - this will take a while since we are calculating cosine similarities on 600k strings
string_grouper = string_grouper.fit()

In [26]:
# Add the grouped strings
companies['deduplicated_name'] = string_grouper.get_groups()